In [1]:
import pyopencl as cl
import xopto

print(f"Successfully imported xopto version: {xopto.__version__}")

print("\n--- Verifying PyOpenCL Installation ---")
platforms = cl.get_platforms()
if not platforms:
    print("!!! ERROR: No OpenCL platforms found. Check Intel OpenCL runtime installation.")
else:
    print(f"Found {len(platforms)} OpenCL platform(s).")
    for i, platform in enumerate(platforms):
        print(f"\n--- Platform {i}: {platform.name} ---")
        devices = platform.get_devices()
        if not devices:
            print("  No devices found on this platform.")
        else:
            print(f"  Found {len(devices)} device(s).")
            for j, device in enumerate(devices):
                print(f"    Device {j}: {device.name}")
                print(f"      Type: {cl.device_type.to_string(device.type)}")
                print(f"      Max Compute Units: {device.max_compute_units}")

Successfully imported xopto version: 0.2.3

--- Verifying PyOpenCL Installation ---
Found 2 OpenCL platform(s).

--- Platform 0: Intel(R) OpenCL Graphics ---
  Found 1 device(s).
    Device 0: Intel(R) Iris(R) Xe Graphics
      Type: ALL | GPU
      Max Compute Units: 96

--- Platform 1: Portable Computing Language ---
  Found 1 device(s).
    Device 0: cpu-haswell-13th Gen Intel(R) Core(TM) i9-13900H
      Type: ALL | CPU
      Max Compute Units: 20


In [2]:
import numpy as np
from matplotlib import pyplot as pp
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# from skin_models import create_hemangioma_model
from skin_models import create_hemangioma_model
from xopto.mcml import mc
from xopto.cl import clinfo

# --- 1. Model and Simulation Setup ---

In [3]:
# # Select the first available OpenCL device (CPU in this case)
# try:
#     device = clinfo.gpus()[0] # pyxopto often labels CPUs as GPUs
# except IndexError:
#     print("Error: No OpenCL device found. Please check your installation.")
#     # Exit or handle error

# print(f"Using OpenCL device: {device.name}")
# print(type(device))

In [4]:
# Select the first available OpenCL CPU device
try:
    device = clinfo.cpus()[0]
except IndexError:
    print("Error: No OpenCL CPU device found. Please check your installation.")
    # Optionally, fallback to GPU
    try:
        device = clinfo.gpus()[0]
        print("Falling back to GPU device.")
    except IndexError:
        raise RuntimeError("No OpenCL devices found at all.")

print(f"Using OpenCL device: {device.name}")
print(type(device))

Using OpenCL device: cpu-haswell-13th Gen Intel(R) Core(TM) i9-13900H
<class 'pyopencl._cl.Device'>


In [5]:
# Define simulation parameters
N_PHOTONS = 20  # Small number for a clear animation
WAVELENGTH = 660 # Red light, for deeper penetration

# Create the 7-layer hemangioma model
layers = create_hemangioma_model(WAVELENGTH)

# Define a simple pencil beam source
# source = mc.mcsource.Pencil(origin=np.array(), direction=np.array())
source = mc.mcsource.line.Line(position=np.array([0, 0, 0]), direction=np.array([0, 0, 1]))

# Configure the Trace detector to capture all photon paths
trace_detector = mc.mctrace.Trace()

# Create the Monte Carlo simulator object
mc_obj = mc.Mc(layers, source, trace=trace_detector, cl_devices=device)

# --- 2. Run the Simulation ---

In [6]:
print(f"Running simulation with {N_PHOTONS} photons...")
trace_res, _, _ = mc_obj.run(N_PHOTONS)
print("Simulation complete.")

Running simulation with 20 photons...


RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'cpu-haswell-13th Gen Intel(R) Core(TM) i9-13900H' on 'Portable Computing Language' at 0x562d1b45ae00>:

Error(s) while linking: 
Cannot find symbol mcsim_pf_sample_angles in kernel library
Device cpu-haswell-13th Gen Intel(R) Core(TM) i9-13900H failed to build the program

(options: -I /home/mikhail/miniforge3/envs/light-in-skin-pyxopto/lib/python3.10/site-packages/pyopencl/cl)